# RNN Implementation with pytorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
n_hidden = 35 # node 수
lr = 0.01 # learning rate
epochs = 1000

string = "hello pytorch. how long can a rnn cell remember?"
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_letters = len(char_list)

In [3]:
# 문장이 input으로 들어왔을 때 이것을 연산 가능한 one-hot-vector로 바꾸는 함수
def string_to_onehot(string):
    start = np.zeros(shape = len(char_list), dtype = int)
    end = np.zeros(shape = len(char_list), dtype = int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape = n_letters, dtype = int)
        zero[idx] = 1
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

In [4]:
# one-hot-vector를 다시 문자로 바꾸는 함수
# 토치 텐서를 입력으로 받아 이를 넘파이 배열로 변환하고, 거기서 1인 지점을 인덱스로 잡아 char_list에서 뽑아낸다
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

### RNN class 정의
one-hot-vector로 변환한 단어 하나를 입력값으로 받고 hidden layer 하나를 통과시켜 결과값을 내는 구조.  
입력값이 들어오면 이전 시간의 은닉층 값과의 조합으로 새로운 은닉층 값을 생성하고, 은닉층에서 결과값을 내는 부분의 연산을 한 번 더 통과해 결과값이 나오게 된다.  
그리고 이전 시간의 은닉층 연산값이 없는 초기의 은닉층 값은 0으로 초기화해야하기 때문에 init_hidden이라는 함수를 만든다.

In [15]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
        
    def forward(self, input, hidden):
        hidden = self.act_fn(self.i2h(input) + self.h2h(hidden))
        output = self.i2o(hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    
    
rnn = RNN(n_letters, n_hidden, n_letters)

### loss func, optimizer 정의

In [16]:
loss_func = nn.MSELoss()  #L2 loss func
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

### Training
1. 학습하고자 하는 문장을 one-ot-vector로 변환한 numpy 배열을 토치 텐서 형태로 바꾼다. 이때 자료형은 연산에서 기본적으로 사용되는 torch.FloatTensor로 지정   
2. 앞서 만든 함수대로 start_token + 문장 + end_token 의 구조를 가진 매트릭스가 생성된다. 학습할 때 시작 토큰이 들어오면 결과로 p가, p가 들어오면 y, y가 들어오면 t가 나오게 된다.  
3. one-hot-vector는 문장에 있는 단어 순서대로 배열되어 있으므로 j번째 인덱스에 해당하는 값이 입력으로 들어오면 j+1번째 인덱스에 해당하는 값이 target이 되면 된다.  
4. 문장 전체를 학습하는 과정은 epochs에 지정한 만큼 반복. 이때 내부적으로 입력값과 목표값의 차이를 계산해 문장 전체에 대한손실을 계산해야 한다. 그런데 문장에 대해 학습할 때 매번 loss를 계산해야하므로 total_loss는 0으로 초기화.  
5. 학습을 시작하려면 rnn 은닉층의 초기값을 지정해야하므로 rnn.init_hidden()함수를 통해 0으로 초기화.

In [17]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()
    
    for j in range(one_hot.size()[0]-1):
        input_ = one_hot[j:j+1,:]
        target = one_hot[j+1]
        
        output, hidden = rnn.forward(input_, hidden)
        loss = loss_func(output.view(-1), target.view(-1))
        total_loss += loss
        input_ = output
        
    total_loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("epohc : %d loss : %f"%(i, total_loss))

epohc : 0 loss : 2.321022
epohc : 100 loss : 0.076385
epohc : 200 loss : 0.036367
epohc : 300 loss : 0.014058
epohc : 400 loss : 0.009746
epohc : 500 loss : 0.007281
epohc : 600 loss : 0.019789
epohc : 700 loss : 0.007365
epohc : 800 loss : 0.003761
epohc : 900 loss : 0.006805


간단한 예제이므로 학습에 사용한 문장을 그대로 테스트에도 사용.  
학습할 때는 단어 하나하나를 다 입력으로 넣어주었지만, 테스트 시에는 첫 글자만 입력으로 전달하고 그 다음부터는 모델에서 나온 결과값을 새로운 입력으로 전달해 첫 글자만으로 전체 문장을 생성해내는지 확인한다.

In [19]:
start = torch.zeros(1, len(char_list))
start[:, -2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ""
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output.data)
        input_ = output
        
print(output_string)

hello eceoececbcroa a.ro c cn  c c   onc c oh  c
